In [15]:
import os
import json
import importlib
import torch

import peripheral_model
import perceptual_model


In [19]:
importlib.reload(peripheral_model)
importlib.reload(perceptual_model)


class Model(torch.nn.Module):
    def __init__(
        self,
        config_model={},
        architecture=[],
        input_shape=[2, 65000, 2],
        config_random_slice={"size": [50, 10000], "buffer": [0, 1000]},
        device=None,
    ):
        """ """
        super().__init__()
        self.input_shape = input_shape
        kwargs_peripheral_model = {
            "sr_input": config_model["kwargs_cochlea"].get("sr_input", None),
            "sr_output": config_model["kwargs_cochlea"].get("sr_output", None),
            "config_cochlear_filterbank": config_model["kwargs_cochlea"].get(
                "config_filterbank", {}
            ),
            "config_ihc_transduction": config_model["kwargs_cochlea"].get(
                "config_subband_processing", {}
            ),
            "config_ihc_lowpass_filter": config_model["kwargs_cochlea"].get(
                "kwargs_fir_lowpass_filter_output", {}
            ),
            "config_anf_rate_level": config_model["kwargs_cochlea"].get(
                "kwargs_sigmoid_rate_level_function", {}
            ),
            "config_anf_spike_generator": config_model["kwargs_cochlea"].get(
                "kwargs_spike_generator_binomial", {}
            ),
            "config_random_slice": config_random_slice,
        }
        assert kwargs_peripheral_model["config_ihc_lowpass_filter"].pop(
            "ihc_filter", True
        )
        # print(self.input_shape, json.dumps(kwargs_peripheral_model, indent=4))
        self.peripheral_model = peripheral_model.PeripheralModel(
            **kwargs_peripheral_model,
        )
        self.perceptual_model = perceptual_model.PerceptualModel(
            architecture=architecture,
            input_shape=self.peripheral_model(torch.zeros(self.input_shape)).shape,
            heads=config_model["n_classes_dict"],
            device=device,
        )

    def forward(self, x):
        """ """
        return self.perceptual_model(self.peripheral_model(x))


dir_model = "../phaselocknet/models/sound_localization/simplified_IHC3000_delayed_integration/arch01"
input_shape = [2, 65000, 2]
config_random_slice = {"size": [50, 10000], "buffer": [0, 1000]}

dir_model = "../phaselocknet/models/spkr_word_recognition/simplified_IHC3000/arch0_0000"
input_shape = [2, 40000]
config_random_slice = {"size": [50, 20000], "buffer": [0, 0]}

dir_model = "../phaselocknet/models/spkr_word_recognition/IHC3000/arch0_0000"
input_shape = [2, 3, 50, 20000]
config_random_slice = {}

dir_model = "../phaselocknet/models/sound_localization/IHC3000_delayed_integration/arch01"
input_shape = [2, 3, 50, 13000, 2]
config_random_slice = {"size": [50, 10000], "buffer": [0, 1000]}

with open(os.path.join(dir_model, "config.json")) as f:
    config_model = json.load(f)
with open(os.path.join(dir_model, "arch.json")) as f:
    architecture = json.load(f)

model = Model(
    config_model=config_model,
    architecture=architecture,
    input_shape=input_shape,
    config_random_slice=config_random_slice,
)

for k, v in model(torch.zeros(model.input_shape)).items():
    print(k, v.shape, v.dtype)


label_loc_int torch.Size([2, 504]) torch.float32
